# Flight Delay Demo - v2 CLI & MLflow

## Install prerequisites

Before running the notebook, make sure the correct versions of these libraries are installed.

In [ ]:
!az extension add -n ml

## Training Script

Let's write our training script to the working directory.

The `sklearn.preprocessing.LabelEncoder` encodes target labels with value between 0 and n_classes-1.

The `sklearn.model_selection.train_test_split` splits arrays or matrices into random train and test subsets

The `sklearn.metrics.accuracy_score` is an accuracy classification score. In multilabel classification, this function computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.

The `sklearn.metrics.confusion_matrix` is compute confusion matrix to evaluate the accuracy of a classification.

The `sklearn.metrics.f1_score` computes the F1 score, also known as balanced F-score or F-measure. The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.

The `sklearn.metrics.precision_score` computes the precision. The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives.

The `sklearn.metrics.recall_score` computes the recall. The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

The `sklearn.metrics.roc_auc_score` computes Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.

The `sklearn.metrics.roc_curve` computes Receiver operating characteristic (ROC).

The contents of the training script are found in train.py

In [ ]:
readfile train.py

## Train with sklearn

The `Pipeline()` function purpose is to assemble several steps that can be cross-validated together while setting different parameters.

The `sklearn.linear_model.LogisticRegression` class implements regularized logistic regression using the ‘liblinear’ library, ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ solvers.

The `sklearn.preprocessing.StandardScaler()` function standardizes features by removing the mean and scaling to unit variance.

In [ ]:
!az ml job create -f train.yml

Then, register and deploy the model in one step with MLflow's deployment client.

## Present scoring service predictions

Let's format our service responses and present them in a suitable way to our end users.

## Deploy to Managed Endpoints

First, create a new directory to hold the configuration files for deploying a managed endpoint.

In [ ]:
import os

managed_endpoints = './managed-endpoints'

# Working directory
if not os.path.exists(managed_endpoints):
    os.makedirs(managed_endpoints)
    
if os.path.exists(os.path.join(managed_endpoints,".amlignore")):
  os.remove(os.path.join(managed_endpoints,".amlignore"))

## Create the endpoint definition

The following file contains the details of the environment to host the model and code. 

Define the endpoint configuration
Specific inputs are required to deploy a model on an online endpoint:

1. Model files.
1. Settings to specify the instance type and scaling capacity.

In [ ]:
%%writefile $managed_endpoints/endpointconfig.yml
name: fd-mlflow-mng-endpoint
type: online
auth_mode: key
traffic:
  blue: 100

deployments:
  #blue deployment
  - name: blue
    model: azureml:fd-mlflow-aks-model:1
    instance_type: Standard_DS3_v2
    scale_settings:
      scale_type: manual
      instance_count: 1
      min_instances: 1
      max_instances: 2

## Deploy your managed online endpoint to Azure

This deployment might take up to 15 minutes, depending on whether the underlying environment or image is being built for the first time. Subsequent deployments that use the same environment will finish processing more quickly.

In [ ]:
!az ml endpoint create -g [your resource group name] -w [your AML workspace name] -n fd-mlflow-mng-endpoint -f ./managed-endpoints/endpointconfig.yml

##  Generate a sample request JSON file

Export some test data to a JSON file we can send to the endpoint.

In [ ]:
%%writefile $managed_endpoints/sample-request.json
{"data": [
[6.0,21.0,6.0,1330.0,1600.0,9.0,150.0,16.0,93.0,745.0,33.64044444,-84.42694444,8.0,40.69249722,-74.16866056,29.0,51148.8,53568.0,2.0,0.0,438.4,451.2,0.0,0.0,30.5,28.5,18.0,15.0,2040.0,1720.0],
[4.0,2.0,3.0,1910.0,2035.0,11.0,85.0,222.0,62.0,361.0,35.87763889,-78.78747222,25.0,39.99798528,-82.89188278,33.0,44928.0,45273.6,0.0,0.0,355.2,438.4,0.0,0.0,23.0,12.5,12.0,1.5,1400.0,680.0],
[1.0,3.0,4.0,935.0,1224.0,16.0,229.0,207.0,78.0,1302.0,39.87195278,-75.24114083,36.0,32.89595056,-97.0372,41.0,33177.6,35596.8,0.0,0.0,156.8,252.8,0.0,0.0,-2.0,6.5,-8.0,-4.5,320.0,440.0],
[4.0,3.0,4.0,1000.0,1252.0,16.0,172.0,207.0,206.0,951.0,39.87195278,-75.24114083,36.0,26.68316194,-80.09559417,7.0,45273.6,44582.4,0.0,4.0,425.6,220.8,0.0,0.0,12.0,28.0,0.5,22.5,640.0,2720.0],
[1.0,21.0,1.0,800.0,1045.0,15.0,105.0,198.0,129.0,589.0,41.979595,-87.90446417,12.0,38.94453194,-77.45580972,43.0,33868.8,34905.6,2.0,0.0,256.0,246.4,56.0,0.0,-7.0,-3.0,-17.0,-13.5,160.0,200.0],
[3.0,12.0,3.0,1640.0,1952.0,5.0,192.0,89.0,101.0,1065.0,40.69249722,-74.16866056,29.0,26.07258333,-80.15275,7.0,41817.6,42508.8,0.0,0.0,336.0,368.0,0.0,0.0,10.0,27.0,0.5,19.5,640.0,2280.0],
[3.0,19.0,3.0,1229.0,1346.0,6.0,77.0,151.0,76.0,214.0,40.77724306,-73.87260917,32.0,38.85208333,-77.03772222,43.0,42854.4,42854.4,22.0,0.0,204.8,307.2,0.0,0.0,10.0,15.0,4.0,6.5,800.0,960.0],
[4.0,18.0,5.0,1210.0,1503.0,4.0,173.0,139.0,169.0,944.0,40.63975111,-73.77892556,32.0,28.42888889,-81.31602778,7.0,47692.8,45964.8,0.0,0.0,524.8,508.8,0.0,0.0,22.5,26.0,5.5,11.5,920.0,1360.0],
[11.0,1.0,6.0,615.0,745.0,9.0,90.0,130.0,18.0,432.0,39.71732917,-86.29438417,13.0,33.64044444,-84.42694444,8.0,36633.6,38016.0,0.0,0.0,297.6,387.2,0.0,0.0,22.0,20.5,5.0,2.0,880.0,720.0],
[11.0,24.0,1.0,936.0,1123.0,8.0,107.0,208.0,77.0,602.0,33.43416667,-112.00805559999999,3.0,39.85840806,-104.6670019,5.0,35942.4,34214.4,0.0,0.0,297.6,291.2,0.0,0.0,27.5,17.0,10.0,-9.5,520.0,280.0]]}

## Invoke the endpoint to score data by using your model

You can use either the invoke command or a REST client of your choice to invoke the endpoint and score against it.

In [ ]:
!az ml endpoint invoke -g [your resource group name] -w [your AML workspace name] -n fd-mlflow-mng-endpoint --request-file ./managed-endpoints/sample-request.json

# Traceability

## Update Model

We can update the model registered during deployment with additional metadata, including the linked dataset.

In [ ]:
! az ml model update -n modelname --tags 'This model was developed by Microsoft to showcase the capabilities of Azure ML.', tags={'title': 'Flight Delay Model' 'datasheet_description':

"""
Last updated: October 2020

Based on dataset from by [Statistical Computing Statistical Graphics](http://stat-computing.org/dataexpo/2009/the-data.html)

""",
    'details': 'This model was developed for Microsoft.',
    'date': 'October 2020, trained on data that cuts off at the end of 2008.', 
    'type': 'Classification model',
    'version': '1.0',
    'help': 'https://www.azure.com/',
    'usecase_primary': 
"""
Developed for Flight Delay Demo.

""",
    'usecase_secondary':
"""
Field demos and marketing.

""",
    'usecase_outofscope':
"""
Do not use for production environments.

""",
    'dataset_description':
"""
The data comes originally from RITA where it is described in detail. You can download the data there, or from the bzipped csv files listed below. These files have derivable variables removed, are packaged in yearly chunks and have been more heavily compressed than the originals.

""",
    'motivation': 'Demo the main features behind the Azure ML Workspace environment',
    'caveats':
"""
"""})